In [1]:
!pip install sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

def train_sbert_embeddings(data):
    """
    SBERT ile film özelliklerinden embedding oluşturur.
    """
    print("SBERT ile vektörler oluşturuluyor...")
    data['features'] = data['movie_genres'] + ' ' + data['director_names']
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = sbert_model.encode(data['features'].tolist(), show_progress_bar=True)
    print("Vektörleşme tamamlandı.")
    return sbert_model, embeddings

def save_model(data, sbert_model, embeddings, model_file):
    """
    Modeli ve veriyi kaydeder.
    """
    print("Model kaydediliyor...")
    with open(model_file, 'wb') as file:
        pickle.dump((data, sbert_model, embeddings), file)
    print(f"Model başarıyla kaydedildi: {model_file}")

def load_model(model_file):
    """
    Modeli ve veriyi yükler.
    """
    print("Model yükleniyor...")
    with open(model_file, 'rb') as file:
        data, sbert_model, embeddings = pickle.load(file)
    print("Model başarıyla yüklendi.")
    return data, sbert_model, embeddings

def recommend_movies(user_input, sbert_model, embeddings, data, top_n=5):
    """
    Kullanıcı girdisine göre film önerisi yapar.
    """
    print("Kullanıcı girdisi işleniyor...")
    user_query = f"{user_input['topic']} {user_input['director']} {user_input['liked_movie']}"
    user_embedding = sbert_model.encode([user_query])

    print("Benzerlik hesaplanıyor...")
    similarities = cosine_similarity(user_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]

    recommended_movies = data.iloc[top_indices][['movie_title', 'movie_genres', 'director_names']]
    return recommended_movies

if __name__ == "__main__":
    input_file = "movies.csv"
    data = pd.read_csv(input_file)
    model_file = "sbert_model.pkl"

    user_input = {
        "topic": "Crime Drama",
        "director": "Rian Johnson",
        "liked_movie": "Ozymandias"
    }

    try:
        data, sbert_model, embeddings = load_model(model_file)
    except FileNotFoundError:
        print("Model bulunamadı, yeni model eğitiliyor...")
        sbert_model, embeddings = train_sbert_embeddings(data)
        save_model(data, sbert_model, embeddings, model_file)

    # Öneri yap
    print("Film önerileri hazırlanıyor...")
    recommendations = recommend_movies(user_input, sbert_model, embeddings, data, top_n=5)

    print("Önerilen Filmler:")
    print(recommendations)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Model yükleniyor...
Model bulunamadı, yeni model eğitiliyor...
SBERT ile vektörler oluşturuluyor...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3874 [00:00<?, ?it/s]

Vektörleşme tamamlandı.
Model kaydediliyor...
Model başarıyla kaydedildi: sbert_model.pkl
Film önerileri hazırlanıyor...
Kullanıcı girdisi işleniyor...
Benzerlik hesaplanıyor...
Önerilen Filmler:
                 movie_title                   movie_genres  \
96107  Finding Steve McQueen             ['Crime', 'Drama']   
94533        The Brotherhood             ['Crime', 'Drama']   
61436             Het vonnis             ['Crime', 'Drama']   
76559                   Joan             ['Crime', 'Drama']   
59324                  Brick  ['Crime', 'Drama', 'Mystery']   

                director_names  
96107  ['Mark Steven Johnson']  
94533          ['Martin Ritt']  
61436         ['Jan Verheyen']  
76559       ['Richard Laxton']  
59324         ['Rian Johnson']  
